<a href="https://colab.research.google.com/github/nesrinemanai/travail.txt/blob/main/hdfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Installation de PySpark
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install pyspark -q


In [ ]:

#  Import des bibliothèqu
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr


In [ ]:

# Création d'une session Spark
spark = SparkSession.builder \
    .appName("Calcul_CA_CR_Benefice") \
    .getOrCreate()


In [ ]:

# Simulation du fichier HDFS (produits.csv)
data = [
    (1, "Iphone", 2500.0, 2300.0, 10, 10),
    (2, "Samsung", 1800.0, 1500.0, 15, 15)
]

colonnes = ["id", "nom", "prix_uv", "prix_ua", "promis", "quantite"]

df = spark.createDataFrame(data, colonnes)

# Affichage du contenu simulé
print("=== Table produits ===")
df.show()


=== Table produits ===
+---+-------+-------+-------+------+--------+
| id|    nom|prix_uv|prix_ua|promis|quantite|
+---+-------+-------+-------+------+--------+
|  1| Iphone| 2500.0| 2300.0|    10|      10|
|  2|Samsung| 1800.0| 1500.0|    15|      15|
+---+-------+-------+-------+------+--------+



In [ ]:

#  Calcul du CA, CR et Bénéfice par produit
resultats = df.withColumn("CA", col("prix_uv") * col("quantite")) \
              .withColumn("CR", col("prix_ua") * col("quantite")) \
              .withColumn("benefice", (col("prix_uv") - col("prix_ua")) * col("quantite"))

print("=== Résultats par produit ===")
resultats.select("id", "nom", "CA", "CR", "benefice").show()


=== Résultats par produit ===
+---+-------+-------+-------+--------+
| id|    nom|     CA|     CR|benefice|
+---+-------+-------+-------+--------+
|  1| Iphone|25000.0|23000.0|  2000.0|
|  2|Samsung|27000.0|22500.0|  4500.0|
+---+-------+-------+-------+--------+



In [ ]:

#  Calcul des totaux
totaux = resultats.agg(
    expr("SUM(prix_uv * quantite) AS CA_total"),
    expr("SUM(prix_ua * quantite) AS CR_total"),
    expr("SUM((prix_uv - prix_ua) * quantite) AS benefice_total")
)

print("=== Totaux ===")
totaux.show()


=== Totaux ===
+--------+--------+--------------+
|CA_total|CR_total|benefice_total|
+--------+--------+--------------+
| 52000.0| 45500.0|        6500.0|
+--------+--------+--------------+



In [ ]:

#  Sauvegarde dans un dossier (simulation HDFS)
# (dans Colab, ce sera juste un répertoire local)
resultats.write.mode("overwrite").csv("/content/hdfs_output", header=True)
print("\n Les résultats sont enregistrés dans le dossier '/content/hdfs_output' (comme dans HDFS).")



 Les résultats sont enregistrés dans le dossier '/content/hdfs_output' (comme dans HDFS).
